## Creating epochs and generating evoked responses (ERP/ERF)

In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [3]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                             session='01',
                             task='audiovisual',
                             run='01',
                             root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1,h_freq=40)
events,event_id=mne.events_from_annotations(raw)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


<ipython-input-3-34cfe67f8f23>:9: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [4]:
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Left': 5,
 'Visual/Right': 6}

In [6]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs = mne.Epochs(raw,
                   events=events,
                   event_id=event_id,
                   tmin=tmin,
                   tmax=tmax,
                   baseline=baseline,
                   preload=True)
epochs

Not setting metadata
Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Loading data for 320 events and 481 original time points ...
0 bad epochs dropped


<Epochs |  320 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~444.8 MB, data loaded,
 'Auditory/Left': 72
 'Auditory/Right': 73
 'Button': 16
 'Smiley': 15
 'Visual/Left': 73
 'Visual/Right': 71>

In [8]:
epochs.plot()

<MNEBrowseFigure size 1920x1001 with 4 Axes>

Dropped 0 epochs: 
Channels marked as bad: ['MEG 2443', 'EEG 053']


Exercise:
* Create epochs starting 250 ms before the stimulus onset and ending 800 ms after stimulus onset, and apply baseline correction with a baseline period ranging from -200 to 0 ms

## Selecting epochs based on experimental conditions

In [9]:
epochs['Auditory/Right']

<Epochs |  73 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~104.0 MB, data loaded,
 'Auditory/Right': 73>

In [10]:
epochs['Auditory']

<Epochs |  145 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~203.4 MB, data loaded,
 'Auditory/Left': 72
 'Auditory/Right': 73>

In [11]:
epochs['Left']

<Epochs |  145 events (all good), -0.299693 - 0.499488 sec, baseline [-0.299693, 0] sec, ~203.4 MB, data loaded,
 'Auditory/Left': 72
 'Visual/Left': 73>

In [13]:
epochs['Visual'].plot_image()

Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"


[<Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>]

Exercise:
* Extract all epochs with a "right" condition, and plot the ERP imgae for the EEG channels

In [15]:
epochs['Right'].copy().pick_types(meg=False,eeg=True).plot_image()

Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


[<Figure size 640x480 with 3 Axes>]

## Saving epochs

In [16]:
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif',overwrite=True)

## Creating evoked data

In [17]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [20]:
evoked_auditory

<Evoked | '0.50 * Auditory/Left + 0.50 * Auditory/Right' (average, N=145), [-0.29969, 0.49949] sec, 366 ch, ~4.6 MB>

In [19]:
evoked_auditory.plot(spatial_colors=True)

<Figure size 640x500 with 6 Axes>

In [22]:
evoked_auditory.plot_topomap(ch_type='mag',times=[0,0.050,0.100,0.150,0.200])

<Figure size 900x195 with 6 Axes>

In [23]:
evoked_auditory.plot_joint(picks='mag')

Projections have already been applied. Setting proj attribute to True.


<Figure size 800x420 with 7 Axes>

In [24]:
mne.viz.plot_compare_evokeds([evoked_auditory,evoked_visual],picks='mag')

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 800x600 with 1 Axes>]

Exercise:
* Plot a GFP comparison for the "Visual/Left" and "Visual/Right" conditions of the EEG data

## Saving evoked data

In [25]:
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                 evoked=[evoked_auditory,evoked_visual])

## Reading evoked data

In [26]:
evokeds = mne.read_evokeds(fname=pathlib.Path('out_data')/'evokeds_ave.fif')
evokeds

Reading out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.50 * Auditory/Left + 0.50 * Auditory/Right)
        0 CTF compensation matrices available
        nave = 145 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
    Found the data of interest:
        t =    -299.69 ...     499.49 ms (0.51 * Visual/Left + 0.49 * Visual/Right)
        0 CTF compensation matrices available
        nave = 144 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied


[<Evoked | '0.50 * Auditory/Left + 0.50 * Auditory/Right' (average, N=145), [-0.29969, 0.49949] sec, 366 ch, ~4.6 MB>,
 <Evoked | '0.51 * Visual/Left + 0.49 * Visual/Right' (average, N=144), [-0.29969, 0.49949] sec, 366 ch, ~4.6 MB>]

In [27]:
evokeds[0]

<Evoked | '0.50 * Auditory/Left + 0.50 * Auditory/Right' (average, N=145), [-0.29969, 0.49949] sec, 366 ch, ~4.6 MB>

In [29]:
evoked = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                         condition='0.50 * Visual/Left + 0.50 * Visual/Right')
evoked

Reading out_data\evokeds_ave.fif ...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active


ValueError: condition "0.50 * Visual/Left + 0.50 * Visual/Right" (average) not found, out of found datasets:
"0.50 * Auditory/Left + 0.50 * Auditory/Right" (average)
"0.51 * Visual/Left + 0.49 * Visual/Right" (average)